# DLPlan: Description Logics State Features for Planning

This notebook aims to give an overview of the main functionality of each of the components of the DLPlan library.

## 0. Installation

In [ ]:
!pip install dlplan==0.2.19 xdot

In [48]:
# Load example data used in the notebook
import requests
domain_data = requests.get("https://raw.githubusercontent.com/rleap-project/dlplan/main/colab/delivery/domain.pddl")
instance_data = requests.get("https://raw.githubusercontent.com/rleap-project/dlplan/main/colab/delivery/instance_1_2.pddl")
with open("/content/domain.pddl", "w") as file:
  file.write(domain_data.text)
with open("/content/instance.pddl", "w") as file:
  file.write(instance_data.text)

## 1. Expand and Visualize First-order State Spaces (state_space component)


In [49]:
from dlplan.state_space import generate_state_space

# 1. Generate state space of Delivery instance with 1 package p and 2 locations a,b
state_space = generate_state_space(
    "/content/domain.pddl",
    "/content/instance.pddl",
    index=0).state_space

In [ ]:
# 2. Visualize state space
from IPython.display import Image
state_space_dot = state_space.to_dot(1)
with open("graph.dot", "w") as file:
    file.write(state_space_dot)
!dot -Tpng graph.dot > graph.png
display(Image('graph.png'))

## 2. Evaluate Description Logics State Features (core component)

In [51]:
from dlplan.core import SyntacticElementFactory

# 1. Parse two features
instance_info = state_space.get_instance_info()
vocabulary_info = instance_info.get_vocabulary_info()
factory = SyntacticElementFactory(vocabulary_info)
undelivered_packages = factory.parse_concept(
    "c_and(c_not(c_equal(r_primitive(at,0,1),r_primitive(at_g,0,1))),c_primitive(package,0))")
num_undelivered_packages = factory.parse_numerical(
    "n_count(c_and(c_not(c_equal(r_primitive(at,0,1),r_primitive(at_g,0,1))),c_primitive(package,0)))")

In [ ]:
# 2. Evaluate the features
initial_state = state_space.get_states()[state_space.get_initial_state_index()]
print("Undelivered packages in initial state:",
      undelivered_packages.evaluate(initial_state))
print("Number of undelivered packages in initial states:",
      num_undelivered_packages.evaluate(initial_state))

# 3. Generate Description Logics State Features (generator component)

In [ ]:
from dlplan.generator import generate_features

# 1. Generate and print feature representations
states = state_space.get_states().values()
feature_reprs = generate_features(factory, list(states), feature_limit=400)
print("\n".join(feature_reprs))

In [54]:
# 2. Analyze the generated features
# Concept feature was not pruned
assert repr(undelivered_packages) in feature_reprs
# Numerical feature was pruned
assert repr(num_undelivered_packages) not in feature_reprs
# Reason for pruning
feature = factory.parse_numerical("n_concept_distance(c_primitive(package,0),r_primitive(at_g,0,1),c_equal(r_primitive(at,0,1),r_primitive(at_g,0,1)))")
assert [num_undelivered_packages.evaluate(state) for state in states] == [feature.evaluate(state) for state in states] and \
        num_undelivered_packages.compute_complexity() >= feature.compute_complexity()


# 4. Example Application: General Policies (policy component)


In [ ]:
from dlplan.policy import PolicyReader, PolicyBuilder

# Visualize state space
display(Image('graph.png'))

# Parse policy from text
builder = PolicyBuilder()
policy = PolicyReader().read(
    """
    (:policy\n
    (:booleans
        (0 "b_empty(r_primitive(carrying,0,1))")
    )
    (:numericals
        (0 "n_count(r_and(r_primitive(at,0,1),r_primitive(at_g,0,1)))")
        (1 "n_concept_distance(c_some(r_inverse(r_primitive(at,0,1)),c_primitive(truck,0)),r_primitive(adjacent,0,1),c_some(r_inverse(r_primitive(at_g,0,1)),c_top))")
        (2 "n_concept_distance(c_some(r_inverse(r_primitive(at,0,1)),c_primitive(truck,0)),r_primitive(adjacent,0,1),c_and(c_all(r_inverse(r_primitive(at_g,0,1)),c_bot),c_some(r_inverse(r_primitive(at,0,1)),c_primitive(package,0))))")
    )
    (:rule (:conditions (:c_b_pos 0) (:c_n_gt 2)) (:effects (:e_b_bot 0) (:e_n_bot 0) (:e_n_dec 2)))
    (:rule (:conditions (:c_b_pos 0) (:c_n_eq 2)) (:effects (:e_b_neg 0) (:e_n_bot 0) (:e_n_bot 1)))
    (:rule (:conditions (:c_b_neg 0) (:c_n_gt 1)) (:effects (:e_b_bot 0) (:e_n_bot 0) (:e_n_dec 1)))
    (:rule (:conditions (:c_b_neg 0) (:c_n_eq 1)) (:effects (:e_b_pos 0) (:e_n_dec 0) (:e_n_bot 1)))
    )
    """,
    builder, factory)

# Classify transitions (s,s') into ``Good'' or ``Bad''
# (state 0, state 1): move t to cell containing undelivered package p when not holding a package is ``Good''
assert policy.evaluate(state_space.get_states()[0], state_space.get_states()[1])
# (state 1, state 0): move t *** away *** from cell containing undelivered package p when not holding a package is ``Bad''
assert not policy.evaluate(state_space.get_states()[1], state_space.get_states()[0])

# 5. Integration into Planning Systems

In [ ]:
from dlplan.core import VocabularyInfo, InstanceInfo, State

# Initialize state language
vocabulary = VocabularyInfo()
predicate_0 = vocabulary.add_predicate("predicate", 2)
predicate_1 = vocabulary.add_predicate("predicate_g", 2)

# Initialize specific instance
instance = InstanceInfo(vocabulary, index=0)
atom_0 = instance.add_atom("predicate", ["object_1", "object_2"])
atom_1 = instance.add_static_atom("predicate_g", ["object_2", "object_1"])

# Map planner state to DLPlan state
state = State(instance, [atom_0], index=0)

print(repr(vocabulary))
print(repr(instance))
print(repr(state))